In [1]:
import sys
sys.path.append('../scripts')
from extract_metadata import extract_metadata,label_feature_split
from course_utils import trainTest
import join_data as jd
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.svm import LinearSVC
from models import evaluate_accuracy
import models
from sklearn.grid_search import GridSearchCV
from sklearn import metrics

In [2]:
cases_df = extract_metadata('../data/merged_caselevel_data.csv')
case_ids = cases_df['caseid']

##TODO: 
- Run on larger data set 
- lower min-n-gram count
- try TF-IDF True or false

In [3]:
X, ordered_case_ids,y = jd.construct_sparse_opinion_matrix(cases_df,'../data/docvec_text',100,100)

shape:  (1403, 2389)
number of cases 1403
total time: 50.4344818592


In [9]:
def train_test_split(X,y, pct_train):
    train_rows = int(pct_train*len(y))
    y_train = np.array(y[:train_rows])
    y_test = np.array(y[train_rows:])
    X_train = X[:train_rows]
    X_test = X[train_rows:]
    case_ids_train = ordered_case_ids[:train_rows]
    case_ids_test = ordered_case_ids[train_rows:]
    return X_train,y_train,case_ids_train,X_test,y_test,case_ids_test

X_train,y_train,case_ids_train,X_test,y_test,case_ids_test = train_test_split(X,y,0.75)

In [10]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(1052, 2389)
(351, 2389)
(1052,)
(351,)


In [11]:
clf = LinearSVC(penalty='l1',random_state=0, dual=False)
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=0, tol=0.0001,
     verbose=0)

In [12]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [13]:
clf.score(X_test,y_test)

0.3903133903133903

In [14]:
evaluate_accuracy(y_train,y_train_pred)

	 	 pred
true 	 	 1 	 2 	 3
	 1 	 464 	 1 	 1
	 2 	 0 	 174 	 1
	 3 	 0 	 1 	 410


0.99619771863117867

In [15]:
evaluate_accuracy(y_test,y_test_pred)

	 	 pred
true 	 	 1 	 2 	 3
	 1 	 84 	 38 	 67
	 2 	 24 	 15 	 26
	 3 	 39 	 20 	 38


0.3903133903133903

##TODO:
- Get test score
- Get baseline accuracy

In [20]:
def optimizeSVM(X_train, y_train, regularization_type='l1'):
    '''
    Creates an SVM classifier trained on the given data with an optimized C parameter.
    Args:
      X_train: A dataframe on which to train the features
      y_train: A dataframe on which to evaluate the training data
      score_func: Scoring function.  Dizzying options here.  Consider:
          metrics.accuracy_score
          metrics.f1_score
    Returns:
      A fitted SVM classifier.
    '''
    
    model_to_set = LinearSVC(penalty=regularization_type,random_state=0, dual=False)
    # consider broadening the param_grid to include different SVM kernels and degrees.  See:
    # http://stackoverflow.com/questions/12632992/gridsearch-for-an-estimator-inside-a-onevsrestclassifier
    param_grid = {'C': [10**i for i in range(-3,7)] + [1e30]}
    model_tuning = GridSearchCV(model_to_set, scoring='f1_weighted',param_grid=param_grid)
    
    model_tuning.fit(X_train, y_train)
    print 'best C param for SVM classifier:', model_tuning.best_params_['C']
    print 'best_score: ', model_tuning.best_score_
        
    #return model_tuning.fit(X_train, y_train)

In [21]:
optimizeSVM(X_train,y_train)

best C param for SVM classifier: 0.01
best_score:  0.47986857994


/Users/205341/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


TODO: Test Accuracy


In [ ]:
def optimizeLogistic(X_train, y_train, score_func=metrics.f1_score, regularization_type='l1'):
    '''
    Creates an SVM classifier trained on the given data with an optimized C parameter.
    Args:
      X_train: A dataframe on which to train the features
      y_train: A dataframe on which to evaluate the training data
      score_func: Scoring function.  Dizzying options here.  Consider:
          metrics.accuracy_score
          metrics.f1_score
    Returns:
      A fitted SVM classifier.
    '''
    
    model_to_set = OneVsRestClassifier(LinearSVC(penalty='l1',random_state=0, dual=False))
    # consider broadening the param_grid
    # http://stackoverflow.com/questions/12632992/gridsearch-for-an-estimator-inside-a-onevsrestclassifier
    param_grid = {'estimator__C': [10**i for i in range(-1,0)] + [1e30]}
    model_tuning = GridSearchCV(model_to_set, param_grid=param_grid,
                             score_func=score_func)
    
    model_tuning.fit(X_train, y_train)
    print 'best C param for LR classifier:', model_tuning.best_params_['estimator__C']
    print 'best params: ', model_tuning.best_params_
    print 'best_score: ', model_tuning.best_score_
        
    #return model_tuning.fit(X_train, y_train)

In [ ]:
def optimizeLogisticRegression(X_train, Y_train, scorer, regularization_type='l2'):
    '''Creates a logisitic regression trainied on the given data with an optimized C parameter.
    Args:
      X_train: A dataframe on which to train the features
      Y_train: A dataframe on which to evaluate the training data
      scorer: A string or a scoring function used to optimize the C hyperparameter. Use 'roc_auc'
        to optimize via roc_auc, and use the following to optimize via F2 score:
        ftwo_scorer = metrics.make_scorer(metrics.fbeta_score, beta=2) 
    Returns:
      A fitted logistic regression classifier.
    '''
    param_grid = {'C': [10**i for i in range(-3,7)] + [1e30]}
    lr_classifier = GridSearchCV(linear_model.LogisticRegression(penalty=regularization_type), 
                                 param_grid=param_grid, scoring=scorer, cv=5)
    print 'best C param for LR classifier:', lr_classifier.best_params_['C']
    return lr_classifier.fit(X_train, Y_train)